In [1]:
from ctrade import *
from plot import *
from datetime import datetime
from indicators import *
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

/Users/giovannid/.virtualenvs/work/lib/python2.7/site-packages/matplotlib/cbook.py:136: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [2]:
p = Poloniex()

In [3]:
major = ['ETH', 'ETC', 'BCN', 'DASH', 'DOGE', 'LTC', 'NXT', 'REP', 'ZEC', 'ARDR', 'XRP', 'STR']

In [4]:
pairs = [i for i in p.currency_pairs if 'USDT_' in i]
pairs += [i for i in p.currency_pairs if any([j for j in major if 'BTC_'+j in i])]

In [ ]:
inf = lambda x: np.isinf(x)

def build_dataset(currency_pairs, days_back, period):
    out = pd.DataFrame()
    for i in currency_pairs:
        print(i)
        t = p.chart(i, days_back, period).df
        t[i+'_PC'] = (t['close'] - t['close'].shift(1))/t['close']
        t = t.rename(columns={'close': i})
        t[i+'_volume'] = t['volume']
        t[i+'_volume_PC'] = (t[i+'_volume'] - t[i+'_volume'].shift(1))/t[i+'_volume']
        t[i+'_volume_PC'] = t[i+'_volume_PC'].fillna(0)
        mask = t[i+'_volume_PC'].apply(inf)
        t.loc[mask, i+'_volume_PC'] = 0
        out = pd.concat([out, t[[i+'_PC', i+'_volume_PC', i, i+'_volume']]], axis=1)
    
    df = out[currency_pairs + [i+'_volume' for i in currency_pairs]]
    out = out[out[i+'_PC'].notnull()]
    return out[[i for i in out.columns if i not in currency_pairs]], df

In [ ]:
df = p.chart('BTC_ETH', 60, '15m').df
df['value'] = (df['open'] + df['close'])*0.5
df2 = p.chart('USDT_BTC', 60, '15m').df
df2['value'] = (df2['open'] + df2['close'])*0.5

In [ ]:
df = moving_average_convergence(df, 'value', 50, 15)
df2 = moving_average_convergence(df2, 'value', 50, 15)

In [ ]:
df, q = tag_ranges(df, 'macd_15-50', (0.03,0.97))
df2, q2 = tag_ranges(df2, 'macd_15-50', (0.03,0.97))

In [ ]:
fig = plt.figure(figsize=(16,6))
fig.subplots_adjust(hspace=0)
ax = fig.add_subplot(211)
_ = plot_fts(df, 'value', ax=ax, since=datetime(2017, 5, 8), plot_args={'color': 'black'})
_ = plot_fts(df, 'value_ewma15', ax=ax, since=datetime(2017, 5, 8), plot_args={'color': 'red'})
_ = plot_fts(df, 'value_ewma50', ax=ax, since=datetime(2017, 5, 8), plot_args={'color': 'yellow'})

ax = fig.add_subplot(212)
_ = plot_fts(df, 'macd_15-50', ax=ax, since=datetime(2017, 5, 8), plot_args={'color': 'green'})
ax.plot(ax.get_xlim(), [q[0],q[0]], '--')
ax.plot(ax.get_xlim(), [q[1],q[1]], '--')

In [ ]:
fig = plt.figure(figsize=(16,6))
fig.subplots_adjust(hspace=0)
ax = fig.add_subplot(211)
_ = plot_fts(df2, 'value', ax=ax, since=datetime(2017, 5, 8), plot_args={'color': 'black'})
_ = plot_fts(df2, 'value_ewma15', ax=ax, since=datetime(2017, 5, 8), plot_args={'color': 'red'})
_ = plot_fts(df2, 'value_ewma50', ax=ax, since=datetime(2017, 5, 8), plot_args={'color': 'yellow'})

ax = fig.add_subplot(212)
_ = plot_fts(df2, 'macd_15-50', ax=ax, since=datetime(2017, 5, 8), plot_args={'color': 'green'})
ax.plot(ax.get_xlim(), [q2[0],q2[0]], '--')
ax.plot(ax.get_xlim(), [q2[1],q2[1]], '--')

In [ ]:
def which_q(df, column, value):

    x = ((df.sort_values(column, ascending=True)[column] - value)>0)*1
    return (x==0).sum()/float(x.shape[0])

In [ ]:
def simulate_sell(x, prices):
    state = ['start']
    profit = ['start']
    sell_price = []
    for index in range(len(x)):
        if state[-1]=='S' and x[index]==1:
            gain = (sell_price - prices[index])/sell_price
            profit.append(gain)
            state.append('C'),
        elif (state[-1]=='C' or state[-1]=='start') and x[index]==-1 :
            state.append('S')
            sell_price = prices[index]

    return state, profit

def simulate_buy(x, prices):
    state = ['start']
    profit = ['start']
    sell_price = []
    for index in range(len(x)):
        if state[-1]=='B' and x[index]==-1:
            gain = (prices[index] - sell_price)/sell_price
            profit.append(gain)
            state.append('C'),
        elif (state[-1]=='C' or state[-1]=='start') and x[index]==1 :
            state.append('B')
            sell_price = prices[index]

    return state, profit

def simulate_buy_sell(x, prices, dates):
    state = ['start']
    profit = ['start']
    sell_price = []
    date = []
    for index in range(len(x)):
        if state[-1]=='B' and x[index]==-1:
            gain = (prices[index] - sell_price)/sell_price
            print 'CB', prices[index], gain
            profit.append(gain)
            state.append('C'),
            date[-1] += [dates[index]]
        elif (state[-1]=='C' or state[-1]=='start') and x[index]==1 :
            state.append('B')
            sell_price = prices[index]
            print 'B', prices[index]
            date.append(['B', dates[index]])
        elif state[-1]=='S' and x[index]==1:
            gain = (sell_price - prices[index])/sell_price
            profit.append(gain)
            state.append('C')
            print 'CS', prices[index], gain
            date[-1] += [dates[index]]
        elif (state[-1]=='C' or state[-1]=='start') and x[index]==-1 :
            state.append('S')
            sell_price = prices[index]
            print 'S', prices[index]
            date.append(['S', dates[index]])

            
    return state, profit, date

In [ ]:
a, b, c = simulate_buy_sell(df['macd_15-50_tag'].tolist(), df['value'].tolist(), df2.index.tolist())

In [ ]:
res = pd.DataFrame(np.array([0] + b[1:])+1, index=range(len(b)), columns=['perf'])
res['pnl'] = res['perf'].cumprod()
plt.plot(range(len(res)), res['pnl'])

In [ ]:
trades = pd.DataFrame(c[:-1], columns=['trade', 'open', 'close'])
trades_sell = trades[trades['trade'] == 'S']
trades_bought = trades[trades['trade'] == 'B']

In [ ]:
trades_sell

In [ ]:
trades_bought

In [ ]:
fig = plt.figure(figsize=(16,6))
# fig.subplots_adjust(hspace=0)
ax = fig.add_subplot(211)
ax = plot_fts(df, 'value', ax=ax, plot_args={'color': 'black'})

ax = ax.twinx()
_ = plot_fts(df, 'macd_15-50', ax=ax, plot_args={'color': 'green'})

ax.plot(ax.get_xlim(), [q2[0],q2[0]], '--')
ax.plot(ax.get_xlim(), [q2[1],q2[1]], '--')


ymin, ymax = ax.get_ylim()[0], ax.get_ylim()[1]

opened = [df.index.get_loc(i) for i in trades_bought['open']]
closed = [df.index.get_loc(i) for i in trades_bought['close']]

for iop, op in enumerate(opened):
    ax.axvspan(op, closed[iop], ymin, ymax, facecolor='g', alpha=.3 )

opened = [df.index.get_loc(i) for i in trades_sell['open']]
closed = [df.index.get_loc(i) for i in trades_sell['close']]
for iop, op in enumerate(opened):
    ax.axvspan(op, closed[iop], ymin, ymax, facecolor='r', alpha=.3 )

In [ ]:
a, b, c = simulate_buy_sell(df2['macd_15-50_tag'].tolist(), df2['value'].tolist(), df2.index.tolist())

In [ ]:
res = pd.DataFrame(np.array([0] + b[1:])+1, index=range(len(b)), columns=['perf'])
res['pnl'] = res['perf'].cumprod()
plt.plot(range(len(res)), res['pnl'])

In [ ]:
trades = pd.DataFrame(c[:-1], columns=['trade', 'open', 'close'])
trades_sell = trades[trades['trade'] == 'S']
trades_bought = trades[trades['trade'] == 'B']

In [ ]:
fig = plt.figure(figsize=(16,6))
# fig.subplots_adjust(hspace=0)
ax = fig.add_subplot(211)
ax = plot_fts(df2, 'value', ax=ax, plot_args={'color': 'black'})

ax = ax.twinx()
_ = plot_fts(df2, 'macd_15-50', ax=ax, plot_args={'color': 'green'})

ax.plot(ax.get_xlim(), [q2[0],q2[0]], '--')
ax.plot(ax.get_xlim(), [q2[1],q2[1]], '--')


ymin, ymax = ax.get_ylim()[0], ax.get_ylim()[1]

opened = [df2.index.get_loc(i) for i in trades_bought['open']]
closed = [df2.index.get_loc(i) for i in trades_bought['close']]

for iop, op in enumerate(opened):
    ax.axvspan(op, closed[iop], ymin, ymax, facecolor='g', alpha=.3 )

opened = [df2.index.get_loc(i) for i in trades_sell['open']]
closed = [df2.index.get_loc(i) for i in trades_sell['close']]
for iop, op in enumerate(opened):
    ax.axvspan(op, closed[iop], ymin, ymax, facecolor='r', alpha=.3 )